# DirectX

In this section we derive the projection matrices for Metal. The basic process is the following.

1. Define the view space orthonormal frame.
2. Define the clip space orthonormal frame compatible with the target canonical view volume. The target clip space should have the same orthonormal frame as the target canonical view volume.
3. Define the target canonical view volume's view volume.
4. Define the frustum parameters for the view space viewing frustum.
5. Using the frustum parameters and the canonical view volume parameters, construct a canonical version of the projection of interest.
6. Construct a coordinate transformation from the view space orthonormal frame to the canonincal view space orthonormal frame.
7. Construct a coordinate transformation from the canonical clip space to the target clip space.
8. Multiply the resulting matrices together in the right order to get the final result.

We now proceed to do this for Metal's left-handed and right-handed view space coordinate frames

In [4]:
import sys
import os

sys.path.insert(0, os.path.abspath('../projection_matrices'))

In [26]:
import sympy
import projection_matrices as pm

In [57]:
def equation_for_display(lhs, rhs) -> sympy.Eq:
    expr_lhs = sympy.Symbol('expr_lhs')
    expr_rhs = sympy.Symbol('expr_rhs')
    expr_lhs = sympy.UnevaluatedExpr(lhs)
    expr_rhs = sympy.UnevaluatedExpr(rhs)
    
    return sympy.Eq(expr_lhs, expr_rhs)

In [58]:
def change_of_orientation_lh_to_rh() -> sympy.Matrix:
    return sympy.Matrix([
        [1, 0,  0, 0],
        [0, 1,  0, 0],
        [0, 0, -1, 0],
        [0, 0,  0, 1]
    ])

def change_of_orientation_rh_to_lh() -> sympy.Matrix:
    return sympy.Matrix([
        [1, 0,  0, 0],
        [0, 1,  0, 0],
        [0, 0, -1, 0],
        [0, 0,  0, 1]
    ])

def change_of_orientation_lh_to_lh() -> sympy.Matrix:
    return sympy.Matrix([
        [1, 0,  0, 0],
        [0, 1,  0, 0],
        [0, 0,  1, 0],
        [0, 0,  0, 1]
    ])

def change_of_orientation_rh_to_rh() -> sympy.Matrix:
    return sympy.Matrix([
        [1, 0,  0, 0],
        [0, 1,  0, 0],
        [0, 0,  1, 0],
        [0, 0,  0, 1]
    ])

## Vector Space Details For The DirectX Left-Handed Orthornormal Frames

The **view space** is a vector space $V_{\text{view}} := (\mathbb{R}^3, O_{\text{view}}, B_{\text{view}})$ with the following properties:

* The underlying vector space is $\mathbb{R}^3$.
* The **origin** of the frame is $O_{\text{view}} := \begin{bmatrix} 0 & 0 & 0 \end{bmatrix}^T$
* The **basis** of the frame is $B_{\text{view}} := \{ \hat{x}, \hat{y}, -\hat{z} \}$ where 
$\hat{x} := \begin{bmatrix} 1 & 0 & 0 \end{bmatrix}^T, \quad 
\hat{y} := \begin{bmatrix} 0 & 1 & 0 \end{bmatrix}^T, \quad 
\hat{z} := \begin{bmatrix} 0 & 0 & 1 \end{bmatrix}^T$. The basis vector $\hat{x}$ points to the right, the basis vector $\hat{y}$ points up, and the basis vector $-\hat{z}$ points into the view volume.
* The orthonormal frame $(O_{\text{view}}, B_{\text{view}})$ has a left-handed orientation.

The **clip space** is a vector space $V_{\text{clip}} := (\mathbb{R}^3, O_{\text{clip}}, B_{\text{clip}})$ with the following properties:

* The underlying vector space is $\mathbb{R}^3$.
* The **origin** of the frame is $O_{\text{clip}} := \begin{bmatrix} 0 & 0 & 0 \end{bmatrix}^T$
* The **basis** of the frame is $B_{\text{clip}} := \{ \hat{x}, \hat{y}, -\hat{z} \}$ where 
$\hat{x} := \begin{bmatrix} 1 & 0 & 0 \end{bmatrix}^T, \quad 
\hat{y} := \begin{bmatrix} 0 & 1 & 0 \end{bmatrix}^T, \quad 
\hat{z} := \begin{bmatrix} 0 & 0 & 1 \end{bmatrix}^T$. The basis vector $\hat{x}$ points to the right, the basis vector $\hat{y}$ points up, and the basis vector $-\hat{z}$ points into the view volume.
* The orthonormal frame $(O_{\text{clip}}, B_{\text{clip}})$ has a left-handed orientation.

The **canonical view volume** is the vector space $V_{\text{cvv}} := (\mathbb{R}^3, O_{\text{cvv}}, B_{\text{cvv}})$ with the following properties:

* The underlying vector space is $\mathbb{R}^3$.
* The **origin** of the frame is $O_{\text{cvv}} := \begin{bmatrix} 0 & 0 & 0 \end{bmatrix}^T$
* The **basis** of the frame is $B_{\text{cvv}} := \{ \hat{x}, \hat{y}, -\hat{z} \}$ where 
$\hat{x} := \begin{bmatrix} 1 & 0 & 0 \end{bmatrix}^T, \quad 
\hat{y} := \begin{bmatrix} 0 & 1 & 0 \end{bmatrix}^T, \quad 
\hat{z} := \begin{bmatrix} 0 & 0 & 1 \end{bmatrix}^T$. The basis vector $\hat{x}$ points to the right, the basis vector $\hat{y}$ points up, and the basis vector $-\hat{z}$ points into the view volume.
* The orthonormal frame $(O_{\text{cvv}}, B_{\text{cvv}})$ has a right-handed orientation.
* The **view volume** is parametrized by $[-1, 1] \times [-1, 1] \times [0, 1]$.

## Derivation Of The Projection Matrices For The Left-Handed View Space Orthonormal Frame

In each case, we map the DirectX view space coordinate system to canonical view space coordinate system, apply the canonical transformation, then apply the clip space coordinate system transformation to get to DirectX's clip space. In the left-handed case, we map DirectX's view space coordinate frame to the canonical view space coordinate frame with an identity map. We map the canonical clip space coordinate frame to DirectX's clip space frame using another identity map. This results in the desired projection matrices.

In [69]:
def perspective_lh_directx():
    l, r, b, t, n, f = sympy.symbols('l r b t n f')
    frustum_bounds = pm.FrustumBounds(l, r, b, t, n, f)
    ndc_bounds = pm.NDCBounds(-1, 1, -1, 1, 0, 1)
    m_per_can_lh_lh = pm.perspective(frustum_bounds, ndc_bounds)

    x_lh_lh = sympy.Matrix.eye(4)
    c_directx = sympy.Matrix.eye(4)
    c_directx_inv = c_directx.inv()
    
    m_per_lh_lh = (x_lh_lh * c_directx_inv) * m_per_can_lh_lh * (c_directx * x_lh_lh)

    return m_per_lh_lh

def perspective_fov_lh_directx():
    aspect, theta_vfov, n, f = sympy.symbols('aspect theta_vfov n f')
    frustum_fov_bounds = pm.FrustumFovBounds(aspect, theta_vfov, n, f)
    ndc_bounds = pm.NDCBounds(-1, 1, -1, 1, 0, 1)
    m_per_fov_can_lh_lh = pm.perspective_fov(frustum_fov_bounds, ndc_bounds)

    x_lh_lh = sympy.Matrix.eye(4)
    c_directx = sympy.Matrix.eye(4)
    c_directx_inv = c_directx.inv()
    
    m_per_fov_lh_lh = (x_lh_lh * c_directx_inv) * m_per_fov_can_lh_lh * (c_directx * x_lh_lh)

    return m_per_fov_lh_lh

def orthographic_lh_directx():
    l, r, b, t, n, f = sympy.symbols('l r b t n f')
    frustum_bounds = pm.FrustumBounds(l, r, b, t, n, f)
    ndc_bounds = pm.NDCBounds(-1, 1, -1, 1, 0, 1)
    m_orth_can_lh_lh = pm.orthographic(frustum_bounds, ndc_bounds)

    x_lh_lh = sympy.Matrix.eye(4)
    c_directx = sympy.Matrix.eye(4)
    c_directx_inv = c_directx.inv()
    
    m_orth_lh_lh = (x_lh_lh * c_directx_inv) * m_orth_can_lh_lh * (c_directx * x_lh_lh)

    return m_orth_lh_lh

The left-handed frustum perspective projection matrix is given by

In [70]:
m_per_directx_lh_sym = sympy.Symbol(r'M^{DirectX}_{per, lh}')
m_per_directx_lh = perspective_lh_directx()
equation_for_display(m_per_directx_lh_sym, m_per_directx_lh)

Eq(M^{DirectX}_{per, lh}, Matrix([
[2*n/(l + r),           0, (l - r)/(l + r),            0],
[          0, 2*n/(b + t), (b - t)/(b + t),            0],
[          0,           0,       f/(f - n), -f*n/(f - n)],
[          0,           0,               1,            0]]))

The left-handed symmetric vertical field of view perspective projection matrix is given by

In [71]:
m_per_fov_directx_lh_sym = sympy.Symbol(r'M^{DirectX}_{per, vfov, lh}')
m_per_fov_directx_lh = perspective_fov_lh_directx()
equation_for_display(m_per_fov_directx_lh_sym, m_per_fov_directx_lh)

Eq(M^{DirectX}_{per, vfov, lh}, Matrix([
[1/(aspect*tan(theta_vfov/2)),                   0,         0,            0],
[                           0, 1/tan(theta_vfov/2),         0,            0],
[                           0,                   0, f/(f - n), -f*n/(f - n)],
[                           0,                   0,         1,            0]]))

The left-handed frustum orthographic projection matrix is given by

In [72]:
m_orth_directx_lh_sym = sympy.Symbol(r'M^{DirectX}_{orth, lh}')
m_orth_directx_lh = orthographic_lh_directx()
equation_for_display(m_orth_directx_lh_sym, m_orth_directx_lh)

Eq(M^{DirectX}_{orth, lh}, Matrix([
[2/(l + r),         0,         0, (l - r)/(l + r)],
[        0, 2/(b + t),         0, (b - t)/(b + t)],
[        0,         0, 1/(f - n),      -n/(f - n)],
[        0,         0,         0,               1]]))

## Vector Space Details For The Metal Right-Handed Orthornormal Frames

The **view space** is a vector space $V_{\text{view}} := (\mathbb{R}^3, O_{\text{view}}, B_{\text{view}})$ with the following properties:

* The underlying vector space is $\mathbb{R}^3$.
* The **origin** of the frame is $O_{\text{view}} := \begin{bmatrix} 0 & 0 & 0 \end{bmatrix}^T$
* The **basis** of the frame is $B_{\text{view}} := \{ \hat{x}, \hat{y}, \hat{z} \}$ where 
$\hat{x} := \begin{bmatrix} 1 & 0 & 0 \end{bmatrix}^T, \quad 
\hat{y} := \begin{bmatrix} 0 & 1 & 0 \end{bmatrix}^T, \quad 
\hat{z} := \begin{bmatrix} 0 & 0 & 1 \end{bmatrix}^T$. The basis vector $\hat{x}$ points to the right, the basis vector $\hat{y}$ points up, and the basis vector $\hat{z}$ points towards the view away from the view volume.
* The orthonormal frame $(O_{\text{view}}, B_{\text{view}})$ has a right-handed orientation.

The **clip space** is a vector space $V_{\text{clip}} := (\mathbb{R}^3, O_{\text{clip}}, B_{\text{clip}})$ with the following properties:

* The underlying vector space is $\mathbb{R}^3$.
* The **origin** of the frame is $O_{\text{clip}} := \begin{bmatrix} 0 & 0 & 0 \end{bmatrix}^T$
* The **basis** of the frame is $B_{\text{clip}} := \{ \hat{x}, \hat{y}, -\hat{z} \}$ where 
$\hat{x} := \begin{bmatrix} 1 & 0 & 0 \end{bmatrix}^T, \quad 
\hat{y} := \begin{bmatrix} 0 & 1 & 0 \end{bmatrix}^T, \quad 
\hat{z} := \begin{bmatrix} 0 & 0 & 1 \end{bmatrix}^T$. The basis vector $\hat{x}$ points to the right, the basis vector $\hat{y}$ points up, and the basis vector $-\hat{z}$ points into the view volume.
* The orthonormal frame $(O_{\text{clip}}, B_{\text{clip}})$ has a left-handed orientation.

The **canonical view volume** is the vector space $V_{\text{cvv}} := (\mathbb{R}^3, O_{\text{cvv}}, B_{\text{cvv}})$ with the following properties:

* The underlying vector space is $\mathbb{R}^3$.
* The **origin** of the frame is $O_{\text{cvv}} := \begin{bmatrix} 0 & 0 & 0 \end{bmatrix}^T$
* The **basis** of the frame is $B_{\text{cvv}} := \{ \hat{x}, \hat{y}, -\hat{z} \}$ where 
$\hat{x} := \begin{bmatrix} 1 & 0 & 0 \end{bmatrix}^T, \quad 
\hat{y} := \begin{bmatrix} 0 & 1 & 0 \end{bmatrix}^T, \quad 
\hat{z} := \begin{bmatrix} 0 & 0 & 1 \end{bmatrix}^T$. The basis vector $\hat{x}$ points to the right, the basis vector $\hat{y}$ points up, and the basis vector $-\hat{z}$ points into the view volume.
* The orthonormal frame $(O_{\text{cvv}}, B_{\text{cvv}})$ has a left-handed orientation.
* The **view volume** is parametrized by $[-1, 1] \times [-1, 1] \times [0, 1]$.

## Derivation Of The Projection Matrices For The Right-Handed View Space Orthonormal Frame

In each case, we map the DirectX view space coordinate system to the canonical view space coordinate system, apply the canonical transformation, then apply the clip space coordinate system transformation to get to DirectX's clip space. In the right-handed case, DirectX's view space coordinate frame and clip space coordinate frame are opposite orientations, so the clip space transformation is a change of orientation matrix. This results in the desired projection matrices.

In [65]:
def perspective_rh_directx():
    l, r, b, t, n, f = sympy.symbols('l r b t n f')
    frustum_bounds = pm.FrustumBounds(l, r, b, t, n, f)
    ndc_bounds = pm.NDCBounds(-1, 1, -1, 1, 0, 1)
    m_per_can_lh_lh = pm.perspective(frustum_bounds, ndc_bounds)

    x_rh_lh = change_of_orientation_rh_to_lh()
    x_lh_lh = change_of_orientation_lh_to_lh()
    c_directx = sympy.Matrix.eye(4)
    c_directx_inv = c_directx.inv()
    
    m_per_rh_lh = (x_lh_lh * c_directx_inv) * m_per_can_lh_lh * (c_directx * x_rh_lh)

    return m_per_rh_lh

def perspective_fov_rh_directx():
    aspect, theta_vfov, n, f = sympy.symbols('aspect theta_vfov n f')
    frustum_fov_bounds = pm.FrustumFovBounds(aspect, theta_vfov, n, f)
    ndc_bounds = pm.NDCBounds(-1, 1, -1, 1, 0, 1)
    m_per_fov_can_lh_lh = pm.perspective_fov(frustum_fov_bounds, ndc_bounds)

    x_rh_lh = change_of_orientation_rh_to_lh()
    x_lh_lh = change_of_orientation_lh_to_lh()
    c_directx = sympy.Matrix.eye(4)
    c_directx_inv = c_directx.inv()
    
    m_per_fov_rh_lh = (x_lh_lh * c_directx_inv) * m_per_fov_can_lh_lh * (c_directx * x_rh_lh)

    return m_per_fov_rh_lh

def orthographic_rh_directx():
    l, r, b, t, n, f = sympy.symbols('l r b t n f')
    frustum_bounds = pm.FrustumBounds(l, r, b, t, n, f)
    ndc_bounds = pm.NDCBounds(-1, 1, -1, 1, 0, 1)
    m_orth_can_lh_lh = pm.orthographic(frustum_bounds, ndc_bounds)

    x_rh_lh = change_of_orientation_rh_to_lh()
    x_lh_lh = change_of_orientation_lh_to_lh()
    c_directx = sympy.Matrix.eye(4)
    c_directx_inv = c_directx.inv()
    
    m_orth_rh_lh = (x_lh_lh * c_directx_inv) * m_orth_can_lh_lh * (c_directx * x_rh_lh)

    return m_orth_rh_lh

The right-handed frustum perspective projection matrix is given by

In [66]:
m_per_directx_rh_sym = sympy.Symbol(r'M^{DirectX}_{per, rh}')
m_per_directx_rh = perspective_rh_directx()
equation_for_display(m_per_directx_rh_sym, m_per_directx_rh)

Eq(M^{DirectX}_{per, rh}, Matrix([
[2*n/(l + r),           0, -(l - r)/(l + r),            0],
[          0, 2*n/(b + t), -(b - t)/(b + t),            0],
[          0,           0,       -f/(f - n), -f*n/(f - n)],
[          0,           0,               -1,            0]]))

The right-handed symmetric vertical field of view perspective projection matrix is given by

In [67]:
m_per_fov_directx_rh_sym = sympy.Symbol(r'M^{DirectX}_{per, vfov, rh}')
m_per_fov_directx_rh = perspective_fov_rh_directx()
equation_for_display(m_per_fov_directx_rh_sym, m_per_fov_directx_rh)

Eq(M^{DirectX}_{per, vfov, rh}, Matrix([
[1/(aspect*tan(theta_vfov/2)),                   0,          0,            0],
[                           0, 1/tan(theta_vfov/2),          0,            0],
[                           0,                   0, -f/(f - n), -f*n/(f - n)],
[                           0,                   0,         -1,            0]]))

The right-handed frustum orthographic projection matrix is given by

In [68]:
m_orth_directx_rh_sym = sympy.Symbol(r'M^{DirectX}_{orth, rh}')
m_orth_directx_rh = orthographic_rh_directx()
equation_for_display(m_orth_directx_rh_sym, m_orth_directx_rh)

Eq(M^{DirectX}_{orth, rh}, Matrix([
[2/(l + r),         0,          0, (l - r)/(l + r)],
[        0, 2/(b + t),          0, (b - t)/(b + t)],
[        0,         0, -1/(f - n),      -n/(f - n)],
[        0,         0,          0,               1]]))